### Importação Bibliotecas

---

In [14]:
import mlflow
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

### Constantes

---

In [5]:
ROOT_PATH = '../../data/'
SEED = 42
TARGET_COL = "default.payment.next.month"


Definir a diretoria onde as experiências são guardadas

In [6]:
from pathlib import Path

uri = "../../mlruns"

Path(uri).mkdir(parents=True, exist_ok=True)

mlflow.set_tracking_uri(uri)

In [7]:
mlflow.set_experiment("Rumos Bank Lending")

2025/04/01 23:50:16 INFO mlflow.tracking.fluent: Experiment with name 'Rumos Bank Lending' does not exist. Creating a new experiment.


<Experiment: artifact_location=('file:c:/Users/PereiraRodrigues/Documents/Data Science Pos Graduate/Oper '
 'ML/OML-trabalho/rumos_bank/notebooks/mlflow/../../mlruns/885472256422112193'), creation_time=1743551416007, experiment_id='885472256422112193', last_update_time=1743551416007, lifecycle_stage='active', name='Rumos Bank Lending', tags={}>

Dataset

In [10]:
df = pd.read_csv(ROOT_PATH + 'lending_data.csv')
df = df.sample(frac=0.1, random_state=SEED)

In [12]:
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = SEED)

In [13]:
X_train = train_set.drop(['default.payment.next.month'], axis = 'columns')
y_train = train_set['default.payment.next.month']

X_test = test_set.drop(['default.payment.next.month'], axis = 1)
y_test = test_set['default.payment.next.month']

### Criar run mlflow

In [15]:
run = mlflow.start_run(run_name="Ml Model Run - 1 - pipeline")
RUN_ID = run.info.run_uuid
RUN_ID

'6c9fccd039e24ae6b8af7f42a99dc0c8'

### Guardar datasets, modelos, artefactos, métricas e parametros da run

In [16]:
# guardarmos o dataset de treino e de teste associado à run
train_dataset = mlflow.data.from_pandas(train_set, source="split_from_full_dataset", targets=TARGET_COL, name="Rumos Bank Lending Dataset")
test_dataset = mlflow.data.from_pandas(test_set, source="split_from_full_dataset", targets=TARGET_COL, name="Rumos Bank Lending Dataset")
mlflow.log_input(train_dataset, context="train")
mlflow.log_input(test_dataset, context="test")

# Guardamos a seed utilizado como parametro
mlflow.log_param("seed", SEED)

c:\Users\PereiraRodrigues\miniconda3\envs\rumos_bank_lending\Lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'split_from_full_dataset'. Exception: 
  return _dataset_source_registry.resolve(
c:\Users\PereiraRodrigues\miniconda3\envs\rumos_bank_lending\Lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
c:\Users\PereiraRodrigues\miniconda3\envs\rumos_bank_lending\Lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'split_from_full_dataset'. Exception: 
  return _dataset_source_registry.resolve(
c:\Users\PereiraRodrigue

42